In [8]:
from UserItemData import UserItemData
from MovieData import MovieData
from RandomPredictor import RandomPredictor
from Recommender import Recommender
from AveragePredictor import AveragePredictor
from ViewsPredictor import ViewsPredictor
from STDPredictor import STDPredictor
from ItemBasedPredictor import ItemBasedPredictor

# 1 Branje ocen
""" uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())
uim = UserItemData('data/user_ratedmovies.dat',  '12.1.2007', '16.2.2008', 100)
print(uim.nratings()) """

# 2 Branje filmov
""" md = MovieData('data/movies.dat')
print(md.get_title(1)) """

# 3 Naključni prediktor
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item])) """

# 4 Priporočanje
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """

# 5 Napovedovanje s povprečjem
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
avg_predictor = AveragePredictor(b=0)
avg_predictor.fit(uim)
predictions = avg_predictor.predict(78)
for movie_id, predicted_rating in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), predicted_rating)) """

# 6 Priporočanje najbolj gledanih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
views_predictor = ViewsPredictor()
views_predictor.fit(uim)
predictions = views_predictor.predict()
for movie_id, view_count in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), view_count)) """

# 7 Priporočanje kontroverznih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = STDPredictor(100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """


import csv
from collections import defaultdict

def count_movies_rated_by_users(file_path):
    # Dictionary to store user ID and their corresponding count of rated movies
    user_movie_counts = defaultdict(int)
    
    # Open and read the file
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')  # Adjust delimiter if needed
        
        # Skip the header
        next(reader, None)
        
        # Process each row in the file
        for row in reader:
            user_id = int(row[0])  # Assuming first column is user ID
            user_movie_counts[user_id] += 1
    
    # Print the results
    for user_id, movie_count in user_movie_counts.items():
        if user_id==78:
            print(f"User {user_id} rated {movie_count} movies.")

#count_movies_rated_by_users('data/user_ratedmovies.dat')

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
import numpy as np
from collections import defaultdict

class ItemBasedPredictor1:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold
        self.similarities = {}
        self.movie_ratings = {}
        self.global_avg = 0
        self.testCounter = 0;

    def fit(self, X):
        self.movie_ratings = defaultdict(dict)  # {movie_id: {user_id: rating}}
        for user_id, movie_id, rating, _ in X.data:
            #print(user_id, movie_id)
            self.movie_ratings[movie_id][user_id] = rating
            #if user_id == 42059:
                #print(self.movie_ratings[movie_id][user_id])
        
        all_ratings = [rating for movie_ratings in self.movie_ratings.values() for rating in movie_ratings.values()]
        self.global_avg = np.mean(all_ratings)
        
        movie_ids = list(self.movie_ratings.keys())
        self.similarities = defaultdict(dict)
        
        for i, movie1 in enumerate(movie_ids):
            for movie2 in movie_ids[i+1:]:
                sim = self.similarity(movie1, movie2)

                self.testCounter +=1
                
                if sim > 0:
                    self.similarities[movie1][movie2] = sim
                    self.similarities[movie2][movie1] = sim
                    

    def similarity(self, p1, p2):
        # Pridobimo ocene obeh filmov
        users_p1 = self.movie_ratings[p1]
        users_p2 = self.movie_ratings[p2]
        #print(self.movie_ratings)
        # Seznam skupnih uporabnikov
        common_users = set(users_p1.keys()).intersection(users_p2.keys())
        #print(f"common_users between {p1} and {p2}: {common_users}")  # Debugging line

        if len(common_users) < self.min_values:
            return 0

        # Pridobimo ocene za skupne uporabnike
        ratings_p1 = np.array([users_p1[user] for user in common_users])
        ratings_p2 = np.array([users_p2[user] for user in common_users])

        # Povprečne ocene uporabnikov
        avg_p1 = np.mean(ratings_p1)
        avg_p2 = np.mean(ratings_p2)

        # Odštejemo povprečja, da "centriramo" ocene
        centered_p1 = ratings_p1 - avg_p1
        centered_p2 = ratings_p2 - avg_p2

        # Izračunamo števnik (dot produkt)
        numerator = np.dot(centered_p1, centered_p2)

        # Izračunamo imenovalec (norme)
        denominator = np.sqrt(np.sum(centered_p1**2)) * np.sqrt(np.sum(centered_p2**2))

        if denominator == 0:
            return 0

        # Izračunamo popravljeno cosinusno podobnost
        sim = numerator / denominator
        #print(f"similarity between {p1} and {p2}: {sim}")  # Izpis podobnosti

        return sim if sim >= self.threshold else 0


    def predict(self, user_id):
        user_id = 42059 
        # Predict ratings for all movies for the given user
        predicted_ratings = {}
        
       # Find all movies in the system
        rated_movies = self.movie_ratings.keys()

        # Find all movies rated by the user
        user_rated_movies = [
            movie_id for movie_id, user_ratings in self.movie_ratings.items()
            if user_id in user_ratings
        ]
        #print("->", user_rated_movies)
        # Determine unrated movies
        unrated_movies = set(rated_movies) - set(user_rated_movies)


        #print(":", unrated_movies)
        for movie in unrated_movies:
            # Calculate the predicted rating for each unrated movie
            weighted_sum = 0
            similarity_sum = 0
            
            for other_movie, similarity in self.similarities[movie].items():
                #print(other_movie, similarity)
                if user_id in self.movie_ratings[other_movie]:
                    print(other_movie)
                    rating = self.movie_ratings[other_movie][user_id]
                    weighted_sum += similarity * (rating - np.mean(list(self.movie_ratings[other_movie].values())))
                    similarity_sum += abs(similarity)
                    

            # Add the global average rating as a baseline if no similar movies are found
            if similarity_sum > 0:
                predicted_rating = np.mean([np.mean(list(self.movie_ratings[other_movie].values())) for other_movie in self.similarities[movie].keys()]) + weighted_sum / similarity_sum
            else:
                predicted_rating = self.global_avg
                #print(similarity_sum)
            
            predicted_ratings[movie] = predicted_rating
        
        return predicted_ratings
    
    def get_top_most_similar_pairs(self, n=20):
        #return self.testCounter
        #return len(self.similarities.items())
        sorted_similarities = sorted(
            [(movie1, movie2, sim) for movie1, movie2_dict in self.similarities.items() for movie2, sim in movie2_dict.items()],
            key=lambda x: x[2], reverse=True
        )
        return sorted_similarities[:n]
# 1
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)


""" print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))
print("Predictions for 78: ") """
rec_items = rec.recommend(533, n=15, rec_seen=False)
""" for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """

# 2
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor(min_values=1000)
rp.fit(uim)
top_similar_pairs = rp.get_top_most_similar_pairs(n=20)
print(top_similar_pairs)
for movie1, movie2, similarity in top_similar_pairs:
    print(f"Film1: {md.get_title(movie1)}, Film2: {md.get_title(movie2)}, podobnost: {similarity}") 

AttributeError: 'ItemBasedPredictor' object has no attribute 'get_top_most_similar_pairs'